# Week 11: Time Series Forecasting for Portfolio Management Optimization

## Fetch Data

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy.optimize import minimize

# Fetch historical data
def fetch_data(tickers, start="2015-01-01", end="2025-01-31"):
    data = yf.download(tickers, start=start, end=end)["Adj Close"]
    return data


# Main Execution
tickers = ["TSLA", "BND", "SPY"]
data = fetch_data(tickers)


## Pre-process

In [ ]:
# Data Preprocessing
def preprocess_data(data):
    data = data.dropna()
    return data

data = preprocess_data(data)


## EDA

In [ ]:
# Exploratory Data Analysis (EDA)
def plot_closing_prices(data):
    plt.figure(figsize=(12,6))
    for col in data.columns:
        plt.plot(data.index, data[col], label=col)
    plt.legend()
    plt.title("Closing Prices Over Time")
    plt.show()


def calculate_daily_returns(data):
    return data.pct_change().dropna()

def plot_volatility(data):
    volatility = data.pct_change().rolling(window=30).std()
    volatility.plot(figsize=(12,6), title="Rolling 30-day Volatility")
    plt.show()
plot_closing_prices(data)
returns = calculate_daily_returns(data)
plot_volatility(data)

## Portifolio Analysis & Optimization

In [ ]:
def decompose_seasonality(data, ticker):
    decomposition = seasonal_decompose(data[ticker], model='multiplicative', period=252)
    decomposition.plot()
    plt.show()

def fit_arima_model(data, ticker):
    model = auto_arima(data[ticker], seasonal=False, stepwise=True, suppress_warnings=True)
    return model

def forecast_arima(model, steps=252):
    forecast = model.predict(n_periods=steps)
    return forecast

def optimize_portfolio(returns):
    num_assets = len(returns.columns)
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    
    def portfolio_performance(weights, mean_returns, cov_matrix):
        returns = np.dot(weights, mean_returns)
        stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return returns, stddev
    
    def negative_sharpe(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
        returns, stddev = portfolio_performance(weights, mean_returns, cov_matrix)
        return -(returns - risk_free_rate) / stddev
    
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    optimized = minimize(negative_sharpe, weights, args=(mean_returns, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)
    return optimized.x

decompose_seasonality(data, "TSLA")
tsla_arima = fit_arima_model(data, "TSLA")
tsla_forecast = forecast_arima(tsla_arima)

# Portfolio Optimization
optimal_weights = optimize_portfolio(returns)
print("Optimized Portfolio Weights:", optimal_weights)